In [1]:
import pandas as pd
import numpy as np

In [2]:
tr_guild = pd.read_csv("../../train/train_guild.csv")
te_guild = pd.read_csv("../../test/test_guild.csv")
tr_id = pd.read_csv("../../train/train_id.csv")
te_id = pd.read_csv("../../test/test_id.csv")
tr_guild.head()

,guild_id,guild_member_acc_id
0,000b682e9e50904b317c6f4afe17d8ee38b8c339f08bf9...,ba86804424a9a3708b1564a7726783612a75a5b881095e...
1,000f26195f92ca13ec8683a6961a6c31ad07efde0d7b80...,ddea37c9b636c4b1650414052f6ede370481e9e236b193...
2,0015cb37aef2cef211f106244519ea8031c04091c9ac34...,9eee6b10b87dce56b7b1eae9d591e16ec128d0c8376c28...
3,001cbf66fa9c3e0ccead23d1c9cc444cb5a2ea3d5ab874...,25810af9e9cff193561a77da73684a3113c3c1539168b1...
4,001ce79ebfcf328508c8d61c476e5bacb3d84f029b5623...,fa5e9cb3dadb37c4797ca7785ea99558ebef24bcbbf844...


# guild 데이터 전처리
guild 데이터는 acc_id가 중복되므로 합쳐서 처리한다.

In [3]:
to_guild = pd.concat([tr_guild, te_guild]).drop_duplicates().sort_values(by=['guild_id']).reset_index(drop=True)

In [4]:
ids = to_guild['guild_member_acc_id'].str.split(',').sum()
# 길이가 다르다는 것은 중복되는 것이 있다는 뜻.
len(ids), len(set(ids))

(66026, 42085)

### 캐릭터별 데이터가 아닌 계정별 데이터임에 주목.
### 위 결과를 보면 여러 길드에 다수 등장하는 계정이 존재한다.
### 길드를 옮겼거나, 계정 내 다른 캐릭터가 다른 길드에 가입한 것으로 추정.
* guild_id 기준인 데이터를 acc_id 기준으로 바꿔준다.

In [5]:
# key: guild_id
# value: guild_mem_count
guild = {i:j for i, j in zip(to_guild['guild_id'], to_guild['guild_member_acc_id'].str.split(',').str.len())}

In [6]:
dic = {'acc_id': [], 'guild_id': [], 'guild_mem_count': []}
for idx, row in to_guild.iterrows():
    ids = row['guild_member_acc_id'].split(',')
    n = len(ids)
    dic['acc_id'].extend(ids)
    dic['guild_id'].extend([row['guild_id']] * n)
    dic['guild_mem_count'].extend([n] * n)
acc_guild = pd.DataFrame(dic).sort_values(by=list(dic.keys())).reset_index(drop=True)
acc_guild.head()

,acc_id,guild_id,guild_mem_count
0,000201ba3f72429177413620a3d22dfb348f21622fa7b6...,a82b3f2c7272567ff2c25dba0df8d746b255a0658561dd...,9
1,0007838a3f353aa4a4c938d42ee43a2fa51b03557e624a...,dc87ae23d97c54b272abd8ca99d10a5dd630f3956d81a6...,8
2,00099da993f9a00f3c6575d844d4bf79a658d1e5845a8a...,59a8d93c3a4e77fa721cf352faa76b7861aa653faac005...,6
3,000af534394783c3a1d28414ce6adc95af45657e34bf70...,c138c5a6efc4d8ddd95ec80965a0cd1db5396eb4f45ab9...,14
4,000c7d37960d5261d7680320227e07e53580f654cf0322...,7aa46f4e050675383b184faa4d03f7c1188cd370f2ff77...,25


In [7]:
acc_guild_pre = acc_guild.groupby('acc_id').agg({'guild_id': ','.join, 'guild_mem_count': ['sum', 'count', 'max']}).reset_index(drop=False)
acc_guild_pre.columns = ['acc_id', 'guild_id', 'total_guild_mem', 'guild_count', 'max_guild_mem']

In [8]:
acc_guild_pre['mean_guild_mem'] = acc_guild_pre['total_guild_mem'] / acc_guild_pre['guild_count']

max_guild_id 를 만듦으로써 실제 활동하는 길드원들끼리 뭉쳐서 데이터를 확인할 수 있음.

In [9]:
acc_guild_pre['max_guild_id'] = acc_guild_pre['guild_id'].str.split(',').apply(lambda x: max([(guild[i],i) for i in x])[1])

In [10]:
acc_guild_pre.head()

,acc_id,guild_id,total_guild_mem,guild_count,max_guild_mem,mean_guild_mem,max_guild_id
0,000201ba3f72429177413620a3d22dfb348f21622fa7b6...,a82b3f2c7272567ff2c25dba0df8d746b255a0658561dd...,9,1,9,9.0,a82b3f2c7272567ff2c25dba0df8d746b255a0658561dd...
1,0007838a3f353aa4a4c938d42ee43a2fa51b03557e624a...,dc87ae23d97c54b272abd8ca99d10a5dd630f3956d81a6...,8,1,8,8.0,dc87ae23d97c54b272abd8ca99d10a5dd630f3956d81a6...
2,00099da993f9a00f3c6575d844d4bf79a658d1e5845a8a...,59a8d93c3a4e77fa721cf352faa76b7861aa653faac005...,6,1,6,6.0,59a8d93c3a4e77fa721cf352faa76b7861aa653faac005...
3,000af534394783c3a1d28414ce6adc95af45657e34bf70...,c138c5a6efc4d8ddd95ec80965a0cd1db5396eb4f45ab9...,14,1,14,14.0,c138c5a6efc4d8ddd95ec80965a0cd1db5396eb4f45ab9...
4,000c7d37960d5261d7680320227e07e53580f654cf0322...,7aa46f4e050675383b184faa4d03f7c1188cd370f2ff77...,25,1,25,25.0,7aa46f4e050675383b184faa4d03f7c1188cd370f2ff77...


In [11]:
tr_acc_guild = pd.merge(tr_id, acc_guild_pre, how='left')
te_acc_guild = pd.merge(te_id, acc_guild_pre, how='left')
te_acc_guild.head(10)

,acc_id,guild_id,total_guild_mem,guild_count,max_guild_mem,mean_guild_mem,max_guild_id
0,0002a56a036206aea3a6c6ebc985df4a2d1987b562e7f0...,NaN,NaN,NaN,NaN,NaN,NaN
1,0003bfdebe2b5d46217e8fa5ab0f33cf186fb558435f20...,NaN,NaN,NaN,NaN,NaN,NaN
2,0008f5dce798aaac3f0b05f8a4269cf2052c9b8c5bd911...,NaN,NaN,NaN,NaN,NaN,NaN
3,000cacd86bddabebef813351455304df5493cd71a3d1d2...,NaN,NaN,NaN,NaN,NaN,NaN
4,000ef0765596ac6a6a0a62654895e4673825b6102c7f41...,NaN,NaN,NaN,NaN,NaN,NaN
5,0013b91d3408e3bfa7d7b88a0ec49b2fb1cedc4677d9e7...,3bcf5a7bacaddc95e2c7c367cd3148173cc1189aa4c2cb...,45.0,3.0,27.0,15.0,3bcf5a7bacaddc95e2c7c367cd3148173cc1189aa4c2cb...
6,0015a0e5bd1bdd2a539a6e7387baa2b12576a685f21d1b...,3d6e58b07201cee6d818610f5b12bfaa0e61ec66e49a8d...,29.0,1.0,29.0,29.0,3d6e58b07201cee6d818610f5b12bfaa0e61ec66e49a8d...
7,0018858b7f1b9941fd957ead27b3abf97ae1bc21627f0b...,NaN,NaN,NaN,NaN,NaN,NaN
8,001fb818efb64b03b6017990997ab0033665159ba6f485...,NaN,NaN,NaN,NaN,NaN,NaN
9,002005ac069dc0614e87def718023dd0b6cddac93d5393...,NaN,NaN,NaN,NaN,NaN,NaN


# guild 데이터 처리 완료

In [12]:
tr_acc_guild.to_csv('train_guild_pre.csv', index=False)
te_acc_guild.to_csv('test_guild_pre.csv', index=False)